In [ ]:
# ✅ Step 1: Install Demucs and FFmpeg
!pip install -q demucs
!apt -qq install -y ffmpeg

# ✅ Step 2: Imports
import os
import shutil
import re
import subprocess
from google.colab import files
from IPython.display import Audio, display

# ✅ Step 3: Set output directory
OUTPUT_DIR = "/content/separated_audio"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ✅ Step 4: Helper to sanitize filename
def create_safe_filename(filename):
    base, ext = os.path.splitext(filename)
    safe_base = re.sub(r"[^\w\-_.]", "_", base)
    return safe_base + ext

# ✅ Step 5: Audio separation function using Demucs
def separate_audio_colab(input_file):
    print(f"Separating: {input_file}")
    safe_filename = create_safe_filename(input_file)
    safe_input_path = os.path.join(os.getcwd(), safe_filename)

    if safe_filename != input_file:
        shutil.move(input_file, safe_input_path)

    try:
        subprocess.run(
            ["python3", "-m", "demucs", "--two-stems=vocals", "-o", OUTPUT_DIR, safe_input_path],
            check=True
        )
    except subprocess.CalledProcessError as e:
        print("❌ Demucs failed:", e)
        return None, None

    base = os.path.splitext(safe_filename)[0]
    return os.path.join(OUTPUT_DIR, "htdemucs", base), base

# ✅ Step 6: Main function for uploading + renaming + downloading
def process_audio_colab():
    print("📁 Please upload your music file:")
    uploaded = files.upload()
    input_file = list(uploaded.keys())[0]

    sep_dir, base_name = separate_audio_colab(input_file)
    if not sep_dir:
        print("❌ Separation failed.")
        return

    original_wav      = os.path.join(OUTPUT_DIR, f"{base_name}.wav")
    vocals_wav        = os.path.join(OUTPUT_DIR, f"{base_name}_vocals.wav")
    instrumental_wav  = os.path.join(OUTPUT_DIR, f"{base_name}_instrumental.wav")

    # Convert uploaded file to .wav
    subprocess.run(["ffmpeg", "-i", os.path.join(os.getcwd(), create_safe_filename(input_file)), original_wav],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    shutil.move(os.path.join(sep_dir, "vocals.wav"), vocals_wav)
    shutil.move(os.path.join(sep_dir, "no_vocals.wav"), instrumental_wav)

    print("✅ Separation complete. Here's a preview:")

    display(Audio(original_wav))
    display(Audio(vocals_wav))
    display(Audio(instrumental_wav))

    print("⬇️ Downloading files...")
    files.download(original_wav)
    files.download(vocals_wav)
    files.download(instrumental_wav)

# ✅ Step 7: Run
process_audio_colab()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

Saving green-day-wake-me-up-when-september-ends-official-audio-128-ytshorts.savetube.me.mp3 to green-day-wake-me-up-when-september-ends-official-audio-128-ytshorts.savetube.me.mp3
Separating: green-day-wake-me-up-when-september-ends-official-audio-128-ytshorts.savetube.me.mp3
✅ Separation complete. Here's a preview:
Buffered data was truncated after reaching the output size limit.